## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.insert(0,'..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,62.01,96,100,4.21,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.40,100,75,1.14,broken clouds
2,2,Vestmannaeyjar,IS,63.4427,-20.2734,45.09,71,20,11.81,few clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,68.52,77,4,15.52,clear sky
4,4,Cabo San Lucas,MX,22.8909,-109.9124,75.99,53,1,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip?"))
max_temp = float(input("What is your desired maximum temperature for your trip?"))

What is your desired minimum temperature for your trip?72
What is your desired maximum temperature for your trip?75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,73.40,100,75,1.14,broken clouds
29,29,Moron,AR,-34.6534,-58.6198,73.99,53,0,9.22,clear sky
74,74,Nishihara,JP,26.1842,127.7558,73.40,78,40,10.36,scattered clouds
96,96,Sao Filipe,CV,14.8961,-24.4956,72.84,77,5,11.48,clear sky
118,118,Popondetta,PG,-8.7537,148.2534,72.77,99,100,1.61,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                49
City                   49
Country                47
Lat                    49
Lng                    49
Max Temp               49
Humidity               49
Cloudiness             49
Wind Speed             49
Current Description    49
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.40,broken clouds,-21.2000,-175.2000,
29,Moron,AR,73.99,clear sky,-34.6534,-58.6198,
74,Nishihara,JP,73.40,scattered clouds,26.1842,127.7558,
96,Sao Filipe,CV,72.84,clear sky,14.8961,-24.4956,
118,Popondetta,PG,72.77,overcast clouds,-8.7537,148.2534,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping.")
        pass

Hotel not found...skipping.
Hotel not found...skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,73.40,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
29,Moron,AR,73.99,clear sky,-34.6534,-58.6198,Hotel Morón
74,Nishihara,JP,73.40,scattered clouds,26.1842,127.7558,Minshuku Agaihama
96,Sao Filipe,CV,72.84,clear sky,14.8961,-24.4956,Tortuga B&B
118,Popondetta,PG,72.77,overcast clouds,-8.7537,148.2534,Timos Transit House
164,Lugoba,TZ,72.25,light rain,-6.4500,38.3333,
174,Victoria,HK,75.00,overcast clouds,22.2855,114.1577,Mini Hotel Central
176,Ampanihy,MG,72.39,broken clouds,-24.7000,44.7500,Angora Hotel
195,Port Hedland,AU,73.40,clear sky,-20.3167,118.5667,The Esplanade Hotel
217,Kawalu,ID,73.13,overcast clouds,-7.3817,108.2082,KOSAN UNGU UNSIK


In [11]:
hotel_df.count()

City                   47
Country                47
Max Temp               47
Current Description    47
Lat                    47
Lng                    47
Hotel Name             47
dtype: int64

In [12]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!='']

In [15]:
clean_hotel_df.count()

City                   45
Country                45
Max Temp               45
Current Description    45
Lat                    45
Lng                    45
Hotel Name             45
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))